In [1]:
import glob, os, json
import solver
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf
from cell import ConvLSTMCell

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

import time
import math

import traceback

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [3]:
df = pd.read_pickle("../data/largefeatures/large200.pickle")
# # Drop rows with NA
# rowsBefore = df.shape[0]
# df = df.dropna()
# print("Dropped %d rows due to None values" % (rowsBefore - df.shape[0]))

In [26]:
def loadDeepWalkInstance(path):
    file = open(path, "r")
    
    i = 0
    
    nodeCount = None
    shape = None
    
    instance = None
    
    for line in file:
        if i == 0:
            split = line.split(" ")
            nodeCount = int(split[0])
            length = int(split[1])
            
            instance = np.zeros(shape=(nodeCount, length))
        else:
            split = line.split(" ")
            
            node = split[0]
            encoding = np.array(list(map(float, split[1:])))
            
            instance[i - 1] = encoding
            
        i += 1
    
    file.close()
    
    return instance

def loadDeepWalkInstances(path):
    instances = []
    names = []
    for file in glob.glob(path + "*.deep"):
        try:
            instance = loadDeepWalkInstance(file)
            split = os.path.splitext(os.path.splitext(os.path.basename(file))[0])
            modifier = split[1][-1:]
            name = split[0] + modifier

            instances.append(instance)
            names.append(name)
        except:
            traceback.print_exc()
    
    return instances, names

In [27]:
instances, names = loadDeepWalkInstances("../data/deep300/")

In [28]:
len(names)

2901

In [29]:
len(df["name"].unique())

2880

In [4]:
MAX_SIZE = 300
INSTANCE_SIZE = 300

In [31]:
# Merge in DeepWalk data
dwInstances = pd.DataFrame(columns=["name", "deepWalk", "sequenceLength"])
reshapedInstances = []
for index, name in enumerate(names):
    instance = instances[index]
    instance = instance.reshape(-1)
    
    size = instance.shape[0]
    
    if name == "pr2392p":
        continue
    
    if size >= INSTANCE_SIZE * MAX_SIZE:
        print(instances[index].shape)
        print("Instance %s is too large" % (name))
        continue
    
    zeroed = np.zeros((INSTANCE_SIZE * MAX_SIZE))
    zeroed[0: size] = instance
    
#     instance = scale(zeroed.astype('float64')).reshape(MAX_SIZE, MAX_SIZE)
    instance = zeroed.astype('float64').reshape(MAX_SIZE, INSTANCE_SIZE)
        
#     reshapedInstances.append(scale(zeroed.astype('float64')).reshape(MAX_SIZE, MAX_SIZE))
    
#     reshapedInstances.append(instance)
#     instance = scale(instance.astype('float64'),axis=1)
    dwInstances = dwInstances.append(pd.DataFrame([[name, instance, size]], columns=["name", "deepWalk", "sequenceLength"]))
    
dwInstances = dwInstances.reset_index().drop("index", axis=1)
df = pd.merge(df, dwInstances, on="name")
df = df.drop("costs", axis=1)

(400, 300)
Instance rd400p is too large
(442, 300)
Instance pcb442p is too large
(431, 300)
Instance gr431p is too large
(318, 300)
Instance lin318p is too large
(561, 300)
Instance pa561p is too large
(403, 300)
Instance rbg403p is too large
(666, 300)
Instance gr666p is too large
(417, 300)
Instance fl417p is too large
(493, 300)
Instance d493p is too large
(358, 300)
Instance rbg358p is too large
(443, 300)
Instance rbg443p is too large
(323, 300)
Instance rbg323p is too large
(439, 300)
Instance pr439p is too large


In [32]:
len(df["name"].unique())

2818

In [3]:
# df.to_pickle("../data/largefeatures/deep300large200noscaleReshaped.pickle")
# df = pd.read_pickle("../data/largefeatures/deeplarge200.pickle")
# df = pd.read_pickle("../data/largefeatures/deep128large200noscaleReshaped.pickle")
df = pd.read_pickle("../data/largefeatures/deep300large200noscaleReshaped.pickle")

In [6]:
df = df.loc[df["metadata.isAsymmetric"] == False]

In [7]:
len(df["name"].unique())

1724

In [8]:
minCostIndices = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.graspCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].idxmin(axis=1)
# minCostIndices = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].idxmin(axis=1)

In [9]:
import collections
collections.Counter(minCostIndices.tolist())

Counter({'heuristics.antColonyCosts': 82,
         'heuristics.geneticCosts': 1,
         'heuristics.graspCosts': 1298,
         'heuristics.simulatedAnnealingCosts': 10,
         'heuristics.tabuCosts': 333})

In [10]:
inputs = np.array(df["deepWalk"].tolist())
sequenceLengths = np.array(df["sequenceLength"].tolist())

In [11]:
costValues = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.graspCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].values
indexRankings = costValues.argsort()

In [12]:
indexRankings

array([[0, 2, 4, 1, 3],
       [0, 2, 4, 1, 3],
       [0, 2, 4, 1, 3],
       ..., 
       [2, 1, 0, 4, 3],
       [0, 4, 2, 3, 1],
       [1, 3, 2, 0, 4]])

In [13]:
# intLabels = LabelEncoder().fit_transform(minCostIndices).reshape(-1, 1)
# # 5 values for 5 different heuristics
# # Drop grasp from analysis
# outputs = OneHotEncoder(sparse=False, n_values=5).fit_transform(intLabels)

# inputs = df

size = df.shape[0]
# Test data is separated in cleaning stage
trainSize = int(size * 0.75)
validSize = size - trainSize

inputsTrain = inputs[0:trainSize]
lengthsTrain = sequenceLengths[0:trainSize]
outputsTrainUnnorm = indexRankings[0:trainSize]
outputsTrain = normalize(outputsTrainUnnorm)

inputsValid = inputs[trainSize:]
lengthsValid = sequenceLengths[trainSize:]
outputsValidUnnorm = indexRankings[trainSize:]
outputsValid = normalize(outputsValidUnnorm)

In [14]:
EPOCHS = 100

N1 = trainSize
LABEL_COUNT = 5

NODES1 = 512
NODES2 = 512
NODES3 = 256
NODES4 = 256
NODES5 = 128
NODES6 = 128
NODES7 = 64

LSTM_SIZE = 150
LSTM_LAYER_COUNT = 2
LSTM_DROPOUT_PROB = 0.7

ALPHA = 0.001

BATCH_SIZE = 2

STD = 0.1

LEARNING_RATE = 0.00001

In [15]:
# Define the input function for training
inputFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain.astype(np.float32), "length": lengthsTrain.astype(np.int32)}, y=outputsTrainUnnorm.astype(np.float32),
#     batch_size=BATCH_SIZE, num_epochs=EPOCHS, shuffle=True)
    num_epochs=EPOCHS, shuffle=True)

In [16]:
inputsTrain[0].shape

(300, 300)

In [17]:
inputsTrain[0][200]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [18]:
# Define the neural network
def network(xDict, mode):
    x = xDict["input"]
        
    length = xDict["length"]
    
    print(x.shape)
        
    # Batch size, timeseries, shape, channels
    x = tf.reshape(x, shape=[-1, MAX_SIZE, INSTANCE_SIZE, 1])
    
    print(x.shape)

#     if mode != tf.estimator.ModeKeys.PREDICT:
#         x = tf.nn.dropout(x, LSTM_DROPOUT_PROB)
 
#     with tf.variable_scope('lstm1'):
#         initialCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
#     initialCell = tf.contrib.rnn.Conv2DLSTMCell(input_shape=(300, 300), output_channels=LSTM_SIZE, kernel_shape=(5,5))
    cell = ConvLSTMCell([INSTANCE_SIZE], 1, [5])
    lstmOutput, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32, sequence_length=length)
    
    pool1 = tf.layers.max_pooling2d(inputs=lstmOutput, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    
#     lstmOutput, state = tf.nn.dynamic_rnn(initialCell, x, dtype=tf.float32, sequence_length=length)
        
#         lstmOutput, _ = initialCell(x, dtype=tf.float32, sequence_length=length)
#         lstmOutput, _ = initialCell(x, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
#     if mode != tf.estimator.ModeKeys.PREDICT:
#         initialCell = tf.contrib.rnn.DropoutWrapper(initialCell, input_keep_prob=LSTM_DROPOUT_PROB, output_keep_prob=LSTM_DROPOUT_PROB, state_keep_prob=LSTM_DROPOUT_PROB, variational_recurrent=True, input_size=x.shape[2], dtype=tf.float64)
    
#     with tf.variable_scope('lstm2'):
#         secondCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
        
#         lstmOutput, _ = secondCell(lstmOutput, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
            
#     with tf.variable_scope('lstm3'):
#         thirdCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
        
#         lstmOutput, _ = thirdCell(lstmOutput, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
            
#     with tf.variable_scope('lstm4'):
#         fourthCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
        
#         lstmOutput, _ = fourthCell(lstmOutput, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
            
#     with tf.variable_scope('lstm5'):
#         fifthCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
        
#         lstmOutput, _ = fifthCell(lstmOutput, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
            
#     with tf.variable_scope('lstm6'):
#         sixthCell = tf.contrib.rnn.LSTMBlockFusedCell(LSTM_SIZE)
        
#         lstmOutput, _ = sixthCell(lstmOutput, dtype=tf.float32)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             lstmOutput = tf.nn.dropout(lstmOutput, LSTM_DROPOUT_PROB)
        
#         if mode != tf.estimator.ModeKeys.PREDICT:
#             secondCell = tf.contrib.rnn.DropoutWrapper(secondCell, input_keep_prob=LSTM_DROPOUT_PROB, output_keep_prob=LSTM_DROPOUT_PROB, state_keep_prob=LSTM_DROPOUT_PROB, variational_recurrent=True, input_size=initialCell.output_size, dtype=tf.float64)
        
#     stackedLstm = tf.contrib.rnn.MultiRNNCell([initialCell, secondCell])
        
#     lstmOutput, _ = tf.nn.dynamic_rnn(stackedLstm, x, dtype=tf.float64, sequence_length=length)
    
    flatten = tf.contrib.layers.flatten(pool2)
    
    regularizer = tf.contrib.layers.l2_regularizer(scale=ALPHA)
    
    # Hidden fully connected layer
#     layer1 = tf.layers.dense(flatten, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
#     layer2 = tf.layers.dense(layer1, NODES2, kernel_regularizer=regularizer, activation=tf.nn.relu)
#     layer3 = tf.layers.dense(layer2, NODES3, kernel_regularizer=regularizer, activation=tf.nn.relu)
    layer1 = tf.layers.dense(flatten, NODES1, activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, NODES2, activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, NODES3, activation=tf.nn.relu)
    layer4 = tf.layers.dense(layer3, NODES4, activation=tf.nn.relu)
    layer5 = tf.layers.dense(layer4, NODES5, activation=tf.nn.relu)
    layer6 = tf.layers.dense(layer5, NODES6, activation=tf.nn.relu)
    layer7 = tf.layers.dense(layer6, NODES7, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    outLayer = tf.layers.dense(layer7, LABEL_COUNT)
    return outLayer

## Loss Functions

In [19]:
# Kullback-Leibler Divergence, as per https://stackoverflow.com/a/43298483
def klDivergence(p, q):
    pClipped = tf.clip_by_value(p, 1e-10, 1.0)
    qClipped = tf.clip_by_value(q, 1e-10, 1.0)
    return tf.reduce_sum(pClipped * tf.log(pClipped/qClipped))

# Loss function based off of Jensen-Shannon Divergence
def loss(label, prediction):
    mean = 0.5 * (label + prediction)
    return 0.5 * klDivergence(label, mean) + 0.5 * klDivergence(prediction, mean)

def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype))
    return numerator / denominator

def listNetLoss(label, prediction):
    softMaxLabel = tf.nn.softmax(label)
    softMaxPrediction = tf.nn.softmax(prediction)
    return -tf.reduce_mean(softMaxLabel * tf.log(softMaxPrediction))

def listMLE(label, prediction):
    sortedPrediction = tf.gather(prediction, tf.nn.top_k(label, k=5).indices)
    final = tf.log(tf.reduce_sum(tf.exp(sortedPrediction)))
    return tf.reduce_sum(final - sortedPrediction)

def listMLE2Loss(labels, predictions, length, length64):
    i = tf.constant(0, dtype=tf.int32)
    innerSum = tf.constant(0, dtype=tf.float32)
    
    def loop(label, prediction, i, innerSum):
        return tf.add(i, 1), tf.add(innerSum, listMLE2(label, prediction))
    
    cond = lambda i, _: tf.less(i, length)
    operation = lambda i, innerSum: loop(labels[i], predictions[i], i, innerSum)
    result = tf.while_loop(cond, operation, [i, innerSum])

    return result[1]/length64
#     return tf.constant(1.0, dtype=tf.float64) * labels + predictions

def listMLE2(label, prediction):
    # Length of vectors
    k = tf.constant(LABEL_COUNT, dtype=tf.int32)
    
    sortedPrediction = tf.gather(prediction, tf.nn.top_k(label, k=k).indices)
    
    j = tf.constant(0, dtype=tf.int32)
    innerSum = tf.constant(0, dtype=tf.float32)
    cond = lambda j, _: tf.less(j, k)
    operation = lambda j, innerSum: listMLE2Loop(sortedPrediction, j, k, innerSum)
    result = tf.while_loop(cond, operation, [j, innerSum])
    
    print(result[1].shape)
    
    return -result[1]
    
def listMLE2Loop(sortedPrediction, j, k, innerSum):
    return tf.add(j, 1), tf.add(innerSum, listMLE2Inner(sortedPrediction, j, k))

def listMLE2Inner(sortedPrediction, j, k):
    numerator = tf.exp(tf.gather(sortedPrediction, j))
    denominator = tf.reduce_sum(tf.exp(sortedPrediction[j:k]))
    
    return tf.log(numerator/denominator)

# Builds an integer ranking out of a 1-D tensor
def convertPredToRank(prediction):
    return tf.cast(tf.nn.top_k(prediction, k=5).indices, dtype=tf.float32)

## Accuracy Measures

In [20]:
# Accuracy metric using Normalized Discounted Cumulative Gain, as per https://github.com/shiba24/learning2rank/
def ndcg(labels, predictions, k=5):
    topK = tf.nn.top_k(labels, k=5)
    sortedValues = topK.values
    sortedIndices = topK.indices
#         print(labelSorted)
#         labelSorted = sorted(label, reverse=True)
    ideal_dcg = 0
    for i in range(k):
#             ideal_dcg += (2 ** labelSorted[:i] - 1.) / log2(tf.cast(i + 2, tf.float64))
        ideal_dcg += (tf.cast(sortedValues[i] + 1, tf.float32)) / log2(tf.cast(i + 2, tf.float32))
    dcg = 0
#         argsort_indices = np.argsort(predictions)[::-1]
#         argsort_indices = tf.nn.top_k(predictions, k=5).indices
#         print(argsort_indices)
    for i in range(k):
        dcg += (tf.gather(predictions, sortedIndices[i]) + 1) / log2(tf.cast(i + 2, tf.float32))
#         dcg += (predictions[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    return dcg / ideal_dcg

def spearmanCorrelation(label, prediction):
    length = tf.cast(tf.shape(prediction)[0], tf.float32)
    sumVal = tf.reduce_sum(tf.square(tf.subtract(prediction, label)))
    return 1 - 6 * sumVal / (length ** 3 - length)

# Bound Spearman coeff. between 0 and 1
def boundedSpearman(label, prediction):
    return (spearmanCorrelation(label, prediction) + 1.)/2

def top1Match(label, prediction):
    return tf.cast(tf.equal(label[0], prediction[0]), tf.float32)

def top2Match(label, prediction):
    sameFirstOrSecond = tf.logical_or(tf.equal(label[0], prediction[0]), tf.equal(label[1], prediction[1]))
    sameFirstAndSecond = tf.logical_or(tf.equal(label[1], prediction[0]), tf.equal(label[0], prediction[1]))
    return tf.cast(tf.logical_or(sameFirstOrSecond, sameFirstAndSecond), tf.float32)

In [21]:
# # Define the model function (following TF Estimator Template)
# def modelFunc(features, labels, mode):
#     # Build the neural network
#     logits = network(features)
    
# #     resizedLogits = tf.reshape(logits, shape=[-1, MAX_SIZE * MAX_SIZE, 1])
    
#     # Predictions
#     # TODO: Possibly need to change
#     pred_classes = logits
# #     pred_classes = tf.argmax(logits, axis=1)
# #     pred_probas = tf.nn.softmax(logits)
#     pred_probas = tf.nn.sigmoid(logits)
    
#     # If prediction mode, early return
#     if mode == tf.estimator.ModeKeys.PREDICT:
#         return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
#     print(logits.shape)
# #     print(resizedLogits.shape)
#     print(labels.shape)
#     print(pred_classes.shape)
        
#     # Define loss and optimizer
# #     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
# #         logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
#     loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=logits, labels=labels))
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
#     train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
#     # Evaluate the accuracy of the model
# #     acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
#     acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
#     # TF Estimators requires to return a EstimatorSpec, that specify
#     # the different ops for training, evaluating, ...
#     estim_specs = tf.estimator.EstimatorSpec(
#       mode=mode,
#       predictions=pred_classes,
#       loss=loss_op,
#       train_op=train_op,
#       eval_metric_ops={'accuracy': acc_op})

#     return estim_specs

# Define the model function (following TF Estimator Template)
def modelFunc(features, labels, mode):
    # Build the neural network
    logits = network(features, mode)
    
#     resizedLogits = tf.reshape(logits, shape=[-1, MAX_SIZE * MAX_SIZE, 1])
    
    # Predictions
    # TODO: Possibly need to change
#     pred_classes = logits
    pred_classes = tf.map_fn(convertPredToRank, logits)
#     pred_classes = tf.argmax(logits, axis=1)
#     pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # Define loss and optimizer
#     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#         logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
#     loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=logits, labels=labels))
#     loss_op = tf.reduce_mean(loss(labels, logits))
    loss_op = tf.reduce_mean(listNetLoss(labels, logits))
#     loss_map = tf.map_fn(lambda x: listMLE2(x[0], x[1]), (labels, pred_classes), dtype=tf.float64)
#     print(labels.get_shape()[0])
#     labels_length = tf.shape(labels)[0]
#     loss_op = tf.reduce_mean(listMLE2Loss(labels, logits, labels_length, tf.cast(labels_length, dtype=tf.float32)))
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
    optimizer = tf.contrib.opt.NadamOptimizer(learning_rate=LEARNING_RATE)
#     optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
#     acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
#     acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    ndcg_map = tf.map_fn(lambda x: ndcg(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    ndcg_op = tf.metrics.mean(ndcg_map)
    top1_map = tf.map_fn(lambda x: top1Match(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    top1_op = tf.metrics.mean(top1_map)
    top2_map = tf.map_fn(lambda x: top2Match(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    top2_op = tf.metrics.mean(top2_map)
    spearman_map = tf.map_fn(lambda x: boundedSpearman(x[0], x[1]), (labels, pred_classes), dtype=tf.float32)
    acc_op = tf.metrics.mean(spearman_map)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op, 'ndcg': ndcg_op, 'top1Classification': top1_op, 'top2Classification': top2_op})

    return estim_specs

In [22]:
# Build the Estimator
config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
model = tf.estimator.Estimator(modelFunc, config=tf.contrib.learn.RunConfig(session_config=config, save_summary_steps=10000, log_step_count_steps=10000))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7e887fd390>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 10000, '_save_checkpoints_secs': 600, '_log_step_count_steps': 10000, '_session_config': , '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpaa3ok_yc'}


In [23]:
trainResults = []
validResults = []

In [24]:
trainFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain.astype(np.float32), "length": lengthsTrain}, y=outputsTrainUnnorm.astype(np.float32),
    batch_size=BATCH_SIZE, shuffle=False)

validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid.astype(np.float32), "length": lengthsValid}, y=outputsValidUnnorm.astype(np.float32),
    batch_size=BATCH_SIZE, shuffle=False)

for i in range(0, 100):
    model.train(inputFunc, steps=400)
    
    print("Evaluating Train")
    accuracy = model.evaluate(trainFunc)
    trainResults.append(accuracy)
    
    print("Evaluating Valid")
    accuracy = model.evaluate(validFunc)
    validResults.append(accuracy)

(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:loss = 0.321966, step = 1
INFO:tensorflow:loss = 0.308329, step = 101 (198.872 sec)
INFO:tensorflow:loss = 0.306319, step = 201 (197.846 sec)
INFO:tensorflow:Saving checkpoints for 301 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:loss = 0.295496, step = 301 (202.137 sec)
INFO:tensorflow:Saving checkpoints for 400 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.299042.
Evaluating Train
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Starting evaluation at 2018-04-21-13:49:03
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-400
INFO:tensorflow:Finished evaluation at 2018-04-21-13:52:17
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.430317, global_step = 400, loss = 0.298216, ndcg = 0.862179, top1Classification = 0.345708, top2Classification = 0.998453
Evaluatin

INFO:tensorflow:Finished evaluation at 2018-04-21-15:21:04
INFO:tensorflow:Saving dict for global step 2400: accuracy = 0.528577, global_step = 2400, loss = 0.248843, ndcg = 0.892237, top1Classification = 0.277649, top2Classification = 0.766435
Evaluating Valid
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Starting evaluation at 2018-04-21-15:21:07
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-2400
INFO:tensorflow:Finished evaluation at 2018-04-21-15:22:16
INFO:tensorflow:Saving dict for global step 2400: accuracy = 0.552784, global_step = 2400, loss = 0.357851, ndcg = 0.876419, top1Classification = 0.194896, top2Classification = 0.693735
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-2400
INFO:tensorflow:Saving checkpoints for 2401 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:loss = 0.243926, step = 2401
INFO:tensorflow:loss = 0.245007, step = 2501 (196.663 s

INFO:tensorflow:Saving checkpoints for 4705 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:Saving checkpoints for 4800 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.219508.
Evaluating Train
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Starting evaluation at 2018-04-21-17:04:55
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-4800
INFO:tensorflow:Finished evaluation at 2018-04-21-17:08:15
INFO:tensorflow:Saving dict for global step 4800: accuracy = 0.462762, global_step = 4800, loss = 0.217118, ndcg = 0.871075, top1Classification = 0.315545, top2Classification = 0.7471
Evaluating Valid
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Starting evaluation at 2018-04-21-17:08:17
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-4800
INFO:tensorflow:Finished evaluation at 2018-04-21-17:09:24
INFO:tensorflow:Saving dict for global step 4800: accuracy = 0.524478, global_step = 4800, loss = 0.402796, ndcg = 0.869251, top1Cl

INFO:tensorflow:Starting evaluation at 2018-04-21-18:37:02
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-6800
INFO:tensorflow:Finished evaluation at 2018-04-21-18:38:09
INFO:tensorflow:Saving dict for global step 6800: accuracy = 0.541995, global_step = 6800, loss = 0.398773, ndcg = 0.874742, top1Classification = 0.197216, top2Classification = 0.723898
(?, 300, 300)
(?, 300, 300, 1)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpaa3ok_yc/model.ckpt-6800
INFO:tensorflow:Saving checkpoints for 6801 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:loss = 0.211477, step = 6801
INFO:tensorflow:loss = 0.211156, step = 6901 (195.860 sec)
INFO:tensorflow:loss = 0.211272, step = 7001 (200.328 sec)
INFO:tensorflow:loss = 0.211349, step = 7101 (197.646 sec)
INFO:tensorflow:Saving checkpoints for 7103 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow:Saving checkpoints for 7200 into /tmp/tmpaa3ok_yc/model.ckpt.
INFO:tensorflow

KeyboardInterrupt: 

In [ ]:
trainResults

In [47]:
validResults

[{'accuracy': 0.47911829,
  'global_step': 1011,
  'loss': 0.38012153,
  'ndcg': 0.85750318,
  'top1Classification': 0.14153132,
  'top2Classification': 0.675174},
 {'accuracy': 0.46972162,
  'global_step': 2022,
  'loss': 0.39480475,
  'ndcg': 0.85565573,
  'top1Classification': 0.13921113,
  'top2Classification': 0.63109046},
 {'accuracy': 0.46600926,
  'global_step': 3033,
  'loss': 0.39247605,
  'ndcg': 0.8527562,
  'top1Classification': 0.1438515,
  'top2Classification': 0.66589326},
 {'accuracy': 0.4575406,
  'global_step': 4044,
  'loss': 0.39723518,
  'ndcg': 0.85106581,
  'top1Classification': 0.13689095,
  'top2Classification': 0.6403712},
 {'accuracy': 0.47169375,
  'global_step': 5055,
  'loss': 0.39897355,
  'ndcg': 0.85377109,
  'top1Classification': 0.13689095,
  'top2Classification': 0.68213457},
 {'accuracy': 0.461833,
  'global_step': 6066,
  'loss': 0.39418501,
  'ndcg': 0.85060596,
  'top1Classification': 0.12761021,
  'top2Classification': 0.66357309},
 {'accuracy'

In [39]:
# Evaluate the Model
# Define the input function for evaluating
validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid.astype(np.float32), "length": lengthsValid}, y=outputsValidUnnorm.astype(np.float32),
    batch_size=BATCH_SIZE, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(validFunc)

INFO:tensorflow:Starting evaluation at 2018-04-01-13:09:25
INFO:tensorflow:Restoring parameters from /tmp/tmpg3jck0j2/model.ckpt-101100
INFO:tensorflow:Finished evaluation at 2018-04-01-13:09:27
INFO:tensorflow:Saving dict for global step 101100: accuracy = 0.534803, global_step = 101100, loss = 0.317587, ndcg = 0.871996, top1Classification = 0.577726, top2Classification = 0.958237


{'accuracy': 0.53480279,
 'global_step': 101100,
 'loss': 0.31758687,
 'ndcg': 0.87199575,
 'top1Classification': 0.57772624,
 'top2Classification': 0.95823663}

In [40]:
predictions = list(model.predict(validFunc))

INFO:tensorflow:Restoring parameters from /tmp/tmpg3jck0j2/model.ckpt-101100


In [41]:
import collections
collections.Counter(list(map(str, predictions)))

Counter({'[ 2.  1.  3.  4.  0.]': 431})

In [50]:
import collections
collections.Counter(list(map(str, indexRankings)))

Counter({'[0 1 2 3 4]': 4,
         '[0 1 2 4 3]': 3,
         '[0 1 4 2 3]': 1,
         '[0 2 1 3 4]': 1,
         '[0 2 1 4 3]': 100,
         '[0 2 3 4 1]': 4,
         '[0 2 4 1 3]': 196,
         '[0 2 4 3 1]': 22,
         '[0 4 2 1 3]': 1,
         '[0 4 2 3 1]': 1,
         '[1 2 0 4 3]': 2,
         '[1 2 3 0 4]': 2,
         '[1 2 3 4 0]': 1,
         '[1 2 4 0 3]': 4,
         '[1 3 2 0 4]': 1,
         '[2 0 1 3 4]': 5,
         '[2 0 1 4 3]': 198,
         '[2 0 3 1 4]': 3,
         '[2 0 3 4 1]': 12,
         '[2 0 4 1 3]': 198,
         '[2 0 4 3 1]': 195,
         '[2 1 0 4 3]': 4,
         '[2 1 3 4 0]': 6,
         '[2 1 4 0 3]': 20,
         '[2 1 4 3 0]': 5,
         '[2 3 0 4 1]': 2,
         '[2 3 1 4 0]': 4,
         '[2 3 4 0 1]': 5,
         '[2 3 4 1 0]': 9,
         '[2 4 0 1 3]': 195,
         '[2 4 0 3 1]': 177,
         '[2 4 1 0 3]': 68,
         '[2 4 1 3 0]': 101,
         '[2 4 3 0 1]': 42,
         '[2 4 3 1 0]': 49,
         '[3 4 1 2 0]': 1,
      

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [1]:
print("hi")

hi


In [22]:
trainResults[len(trainResults) - 1]

{'accuracy': 0.43051046,
 'global_step': 404100,
 'loss': 0.21140459,
 'ndcg': 0.84055918,
 'top1Classification': 0.31090486,
 'top2Classification': 0.71616393}

In [23]:
validResults[len(validResults) - 1]

{'accuracy': 0.53317863,
 'global_step': 404100,
 'loss': 0.4090046,
 'ndcg': 0.8764475,
 'top1Classification': 0.15777262,
 'top2Classification': 0.77030164}

In [22]:
tf.VERSION

'1.3.0'